# Preprocess data

In [13]:
import json

def load_and_convert_data(input_file):
    vehicles_data = {}

    with open(input_file, 'r') as f:
        for line in f:
            obj = json.loads(line.strip())
            vehicle_id = obj['vehicle_id']
            frame_ids = obj['frame_id']
            timestamps = [ts / 1000 for ts in obj['timestamp']]  # 毫秒转秒
            local_xs = [x * 0.3048 for x in obj['local_x']]  # 英尺转米
            local_ys = [y * 0.3048 for y in obj['local_y']]  # 英尺转米
            v_vel = [vel * 0.681818 for vel in obj['v_vel']]  # 英尺/秒转英里/小时
            lane_ids = obj['lane_id']
            precedings = obj['preceding']
            followings = obj['following']

            vehicles_data[vehicle_id] = {
                'frame_id': frame_ids,
                'timestamp': timestamps,
                'local_x': local_xs,
                'local_y': local_ys,
                'v_vel': v_vel,
                'lane_id': lane_ids,
                'preceding': precedings,
                'following': followings,
            }

    return vehicles_data

input_file = '../trajectories-0400-0415.json'
vehicles_data = load_and_convert_data(input_file)
print("Load complete!")

Load complete!


# Find lane_change event

In [14]:
def detect_lane_changes(vehicles_data, output_file):
    lane_changes = []

    for vehicle_id, data in vehicles_data.items():
        lane_id = data['lane_id']
        timestamps = data['timestamp']
        preceding = data['preceding']
        following = data['following']
        lane_id = data['lane_id']

        for i in range(1, len(lane_id)):
            if lane_id[i] != lane_id[i - 1]:
                lane_changes.append({
                    'vehicle_id': vehicle_id,
                    'before_lane_id': lane_id[i-1],
                    'after_lane_id': lane_id[i],
                    'before_change_timestamp': timestamps[i - 1],
                    'after_change_timestamp': timestamps[i],
                    'before_preceding_id': preceding[i-1],
                    'before_following_id': following[i-1],
                    'after_preceding_id': preceding[i],
                    'after_following_id': following[i]
                })

    with open(output_file, 'w') as f:
        for change in lane_changes:
            json.dump(change, f)
            f.write('\n')

output_file = 'trajectories-0400-0415-lane-change.json'
detect_lane_changes(vehicles_data, output_file)

print('Record complete!')

Record complete!


# Complete the output file

In [15]:
def add_lane_change_details(vehicles_data, lane_changes_file, output_file, delta_t=3):
    detailed_lane_changes = []

    with open(lane_changes_file, 'r') as f:
        for line in f:
            change = json.loads(line.strip())
            vehicle_id = change['vehicle_id']
            before_change = change['before_change_timestamp']
            after_change = change['after_change_timestamp']
            
            vehicle_data = vehicles_data[vehicle_id]
            timestamps = vehicle_data['timestamp']
            v_vel = vehicle_data['v_vel']
            local_y = vehicle_data['local_y']

            start_time = before_change - delta_t / 2
            end_time = after_change + delta_t / 2

            change['timestamp'] = []
            change['vehicle_focus_vel'] = []
            change['vehicle_focus_local_y'] = []
            change['vehicle_before_preceding_vel'] = []
            change['vehicle_before_preceding_local_y'] = []
            change['vehicle_before_following_vel'] = []
            change['vehicle_before_following_local_y'] = []
            change['vehicle_after_preceding_vel'] = []
            change['vehicle_after_preceding_local_y'] = []
            change['vehicle_after_following_vel'] = []
            change['vehicle_after_following_local_y'] = []

            # 变道车速度和local_y
            for i, ts in enumerate(timestamps):
                if start_time <= ts <= end_time:
                    change['timestamp'].append(ts)
                    change['vehicle_focus_vel'].append(v_vel[i])
                    change['vehicle_focus_local_y'].append(local_y[i])
                    
            list_number = len(change['timestamp'])

            # 变道前前后车速度和local_y
            before_preceding_id = change['before_preceding_id']
            before_following_id = change['before_following_id']

            if before_preceding_id != 0:
                before_preceding_data = vehicles_data[before_preceding_id]
                before_preceding_timestamps = before_preceding_data['timestamp']
                before_preceding_v_vel = before_preceding_data['v_vel']
                before_preceding_local_y = before_preceding_data['local_y']
                for i, ts in enumerate(change['timestamp']):
                    this_find = False
                    for j, tss in enumerate(before_preceding_timestamps):
                        if ts == tss:
                            change['vehicle_before_preceding_vel'].append(before_preceding_v_vel[j])
                            change['vehicle_before_preceding_local_y'].append(before_preceding_local_y[j])
                            this_find = True
                    if not this_find:
                        change['vehicle_before_preceding_vel'].append(0)
                        change['vehicle_before_preceding_local_y'].append(0)
            else:
                for i, ts in enumerate(change['timestamp']):
                    change['vehicle_before_preceding_vel'].append(0)
                    change['vehicle_before_preceding_local_y'].append(0)

            if before_following_id != 0:
                before_following_data = vehicles_data[before_following_id]
                before_following_timestamps = before_following_data['timestamp']
                before_following_v_vel = before_following_data['v_vel']
                before_following_local_y = before_following_data['local_y']
                for i, ts in enumerate(change['timestamp']):
                    this_find = False
                    for j, tss in enumerate(before_following_timestamps):
                        if ts == tss:
                            change['vehicle_before_following_vel'].append(before_following_v_vel[j])
                            change['vehicle_before_following_local_y'].append(before_following_local_y[j])
                            this_find = True
                    if not this_find:
                        change['vehicle_before_following_vel'].append(0)
                        change['vehicle_before_following_local_y'].append(0)
            else:
                for i, ts in enumerate(change['timestamp']):
                    change['vehicle_before_following_vel'].append(0)
                    change['vehicle_before_following_local_y'].append(0)
                    
            # 变道后前后车速度和local_y
            after_preceding_id = change['after_preceding_id']
            after_following_id = change['after_following_id']

            if after_preceding_id != 0:
                after_preceding_data = vehicles_data[after_preceding_id]
                after_preceding_timestamps = after_preceding_data['timestamp']
                after_preceding_v_vel = after_preceding_data['v_vel']
                after_preceding_local_y = after_preceding_data['local_y']
                for i, ts in enumerate(change['timestamp']):
                    this_find = False
                    for j, tss in enumerate(after_preceding_timestamps):
                        if ts == tss:
                            change['vehicle_after_preceding_vel'].append(after_preceding_v_vel[j])
                            change['vehicle_after_preceding_local_y'].append(after_preceding_local_y[j])
                            this_find = True
                    if not this_find:
                        change['vehicle_after_preceding_vel'].append(0)
                        change['vehicle_after_preceding_local_y'].append(0)
            else:
                for i, ts in enumerate(change['timestamp']):
                    change['vehicle_after_preceding_vel'].append(0)
                    change['vehicle_after_preceding_local_y'].append(0)

            if after_following_id != 0:
                after_following_data = vehicles_data[after_following_id]
                after_following_timestamps = after_following_data['timestamp']
                after_following_v_vel = after_following_data['v_vel']
                after_following_local_y = after_following_data['local_y']
                for i, ts in enumerate(change['timestamp']):
                    this_find = False
                    for j, tss in enumerate(after_following_timestamps):
                        if ts == tss:
                            change['vehicle_after_following_vel'].append(after_following_v_vel[j])
                            change['vehicle_after_following_local_y'].append(after_following_local_y[j])
                            this_find = True
                    if not this_find:
                        change['vehicle_after_following_vel'].append(0)
                        change['vehicle_after_following_local_y'].append(0)            
            else:
                for i, ts in enumerate(change['timestamp']):
                    change['vehicle_after_following_vel'].append(0)
                    change['vehicle_after_following_local_y'].append(0)
            

            detailed_lane_changes.append(change)

            if (len(detailed_lane_changes)) % 400 == 0:
                print(f"Processed {len(detailed_lane_changes)} lane changes")

    with open(output_file, 'w') as f:
        for change in detailed_lane_changes:
            json.dump(change, f)
            f.write('\n')

detailed_output_file = 'trajectories-0400-0415-detailed-lane-change.json'
add_lane_change_details(vehicles_data, 'trajectories-0400-0415-lane-change.json', detailed_output_file)

print('Detail record complete!')

Processed 400 lane changes
Processed 800 lane changes
Detail record complete!
